# Импорт

In [ ]:
import sys
sys.path.append('../')
import momepy
import folium
import pyproj
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
import graphbuilder_v2
import geopandas as gpd
from availability_estimation import *

from pprint import pprint
from pyproj import Transformer

# Примеры работы

## Загрузка графа

In [ ]:
# В качестве начальных данных подается gpd.GeoDataFrame с Polygon, по которому нужно скачать граф
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326)
# city = ox.geocode_to_gdf('R1281563', by_osmid=True) # тестовый мини-город, чтобы долго не грузить

# city = ox.geocode_to_gdf('R1572051', by_osmid=True)

graph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636, retain_all=False)

In [ ]:
# Загрузка локального графа
graph = nx.read_graphml('data/graphml/graph.graphml')

In [ ]:
# Перевод edges:list -> str
graph = graphbuilder_v2.convert_list_attr_to_str(graph)



In [ ]:
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

In [ ]:
nx.write_graphml(graph, 'data/graphml/graph.graphml')

## Создание карты по каркасу

In [ ]:
import folium
from pyproj import Transformer

# Инициализация трансформера для преобразования из EPSG:32636 в EPSG:4326
transformer = Transformer.from_crs("epsg:32636", "epsg:4326")

# Выделение дорог с федеральным/региональным статусом
e = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') in([1, 2])]

m = folium.Map(tiles='CartoDB Dark_Matter')

# Создание FeatureGroup для слоев
lines_layer = folium.FeatureGroup(name='Lines', show=False)
nodes_layer = folium.FeatureGroup(name='Nodes', show=False)
exit_nodes_layer = folium.FeatureGroup(name='Exit Nodes', show=False)

# Преобразование и отображение узлов и линий
for u, v, k in e:
    start_y, start_x = graph.nodes[u]['y'], graph.nodes[u]['x']
    end_y, end_x = graph.nodes[v]['y'], graph.nodes[v]['x']
    
    # Преобразование координат
    start_lat, start_lon = transformer.transform(start_x, start_y)
    end_lat, end_lon = transformer.transform(end_x, end_y)
    
    # Создание линии
    line_coords = [
        [start_lat, start_lon],
        [end_lat, end_lon]
    ]
    folium.PolyLine(locations=line_coords, color='#AAFF01').add_to(lines_layer)
    
    # Условие для цвета CircleMarker и создание tooltip
    start_color = '#FE4F19' if graph.nodes[u].get('exit') == 1 else '#13D1FF'
    end_color =   '#FE4F19' if graph.nodes[v].get('exit') == 1 else '#13D1FF'
    
    start_tooltip = f"Exit: {graph.nodes[u].get('exit')}, Reg1: {graph.nodes[u].get('reg_1')}, Reg2: {graph.nodes[u].get('reg_2')}"
    end_tooltip =   f"Exit: {graph.nodes[v].get('exit')}, Reg1: {graph.nodes[v].get('reg_1')}, Reg2: {graph.nodes[v].get('reg_2')}"
    
    # Добавление CircleMarker для начального узла
    folium.CircleMarker(
        location=[start_lat, start_lon],
        radius=2,
        color=start_color,
        fill=True,
        opacity=1,
        fill_color=start_color,
        tooltip=start_tooltip
    ).add_to(exit_nodes_layer if graph.nodes[u].get('exit') == 1 else nodes_layer)
    
    # Добавление CircleMarker для конечного узла
    folium.CircleMarker(
        location=[end_lat, end_lon],
        radius=2,
        color=end_color,
        fill=True,
        opacity=1,
        fill_color=end_color,
        tooltip=end_tooltip
    ).add_to(exit_nodes_layer if graph.nodes[v].get('exit') == 1 else nodes_layer)

# Добавление слоев на карту
lines_layer.add_to(m)
nodes_layer.add_to(m)
exit_nodes_layer.add_to(m)

# Отображение границы полигона (например, города)
city.boundary.explore(m=m, color='#AA00FF', name='Cities', opacity=0.8)

# Добавление контроля слоев
folium.LayerControl().add_to(m)

# Сохранение карты
# Путь проверьте!!!
m.save('../data/html/rostov_graphbuilder2.html')

## Перевод graph -> GeoDataFrame

### Конвертация str -> geometry

In [ ]:
# Конвертация str -> geometry
graph = graphbuilder_v2.convert_geometry_from_wkt(graph)

In [ ]:
# Преобразуем граф в GeoDataFrames
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

## Вывод на карту определенный тип reg_status дороги

In [ ]:
# Разница между предыдущим и этим методом:
# Вариант выше рисовал узлы и ребра графа. Этот вариант рисует геометрию

edges = edges.to_crs(epsg=4326)
edges_filtered = edges[edges['reg'].isin([1, 2])]

m = folium.Map()

# Определение цветов для разных значений reg
color_map = {1: 'red', 2: 'blue', 3: 'green'}

# Добавление рёбер на карту
for _, row in edges_filtered.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color_map[row['reg']]: {'color': color},
        tooltip=f"highway: {row['highway']}, ref: {row['ref']}, reg: {row['reg']}"
    ).add_to(m)

# Отображение карты
m.save('../data/html/roads.html')

## Проецирование точек на узлы графа

In [ ]:
# Пример проецирования точек на граф. Важно: пример для Ленинградской области и Санкт-Петербурга

lomonosov = ox.geocode_to_gdf('N411691832', by_osmid=True).to_crs(epsg=32636)[['geometry','name']]
lomonosov.loc[0, 'name'] = 'Ломоносов'

city_points = gpd.read_file('../data/geojsons/СНП ЛО.geojson')
rr_points = gpd.read_file('../data/geojsons/ЖД остановки.geojson').to_crs(epsg=32636)

cities = ["приозерск", "кировск", "кингисепп", "луга", 
          "лодейное поле", "гатчина", "тихвин", "тосно", 
          "выборг", "бокситогорск", "всеволожск", "волосово", 
          "волхов", "сосновый бор", "сланцы", "подпорожье", "кириши"]

# Выделение административных центров
city_points = city_points[
    (city_points['name'].str.lower().isin(cities)) &
    (city_points['rural settlement'].str.contains('административный центр'))
].sort_values('name')

city_points = city_points.to_crs(epsg=32636)
city_points = pd.concat([city_points, lomonosov]).reset_index(drop=True)

nodes_fil = nodes[(nodes['reg_1'] == True) | (nodes['reg_2'] == True)]
graph = graphbuilder_v2.assign_city_names_to_nodes(city_points, nodes_fil, graph, name_attr='city_name', node_id_attr='nodeID', name_col='name', max_distance=1200)
graph = graphbuilder_v2.assign_city_names_to_nodes(rr_points, nodes_fil, graph, name_attr='rr_name', node_id_attr='nodeID', name_col='NAME', max_distance=1200)

In [ ]:
import folium
from pyproj import Transformer

# Инициализация трансформера для преобразования из EPSG:32636 в EPSG:4326
transformer = Transformer.from_crs("epsg:32636", "epsg:4326")

# Инициализация карты
m = folium.Map(tiles='CartoDB Dark_Matter')

# Создание FeatureGroup для слоев
nodes_city_layer = folium.FeatureGroup(name='Nodes with City Names', show=False)
nodes_rr_layer = folium.FeatureGroup(name='Nodes with RR Names', show=False)
nodes_default_layer = folium.FeatureGroup(name='Other Nodes', show=False)
lines_layer_reg1 = folium.FeatureGroup(name='Lines reg 1', show=False)
lines_layer_reg2 = folium.FeatureGroup(name='Lines reg 2', show=False)
exit_nodes_layer = folium.FeatureGroup(name='Exit Nodes', show=False)

# Преобразование и отображение узлов и линий
for u, v, k, d in graph.edges(data=True, keys=True):
    start_y, start_x = graph.nodes[u]['y'], graph.nodes[u]['x']
    end_y, end_x = graph.nodes[v]['y'], graph.nodes[v]['x']
    
    # Преобразование координат
    start_lat, start_lon = transformer.transform(start_x, start_y)
    end_lat, end_lon = transformer.transform(end_x, end_y)
    
    # Создание линии
    line_coords = [
        [start_lat, start_lon],
        [end_lat, end_lon]
    ]
    
    if d.get('reg') == 1:
        folium.PolyLine(locations=line_coords, color='blue').add_to(lines_layer_reg1)
    elif d.get('reg') == 2:
        folium.PolyLine(locations=line_coords, color='green').add_to(lines_layer_reg2)
    
    # Проверка и добавление узлов
    for node, lat, lon in [(u, start_lat, start_lon), (v, end_lat, end_lon)]:
        if graph.nodes[node].get('exit') == 1:
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='#FE4F19',
                fill=True,
                fill_color='#FE4F19',
                tooltip=f"Exit: {graph.nodes[node].get('exit')}, Reg1: {graph.nodes[node].get('reg_1')}, Reg2: {graph.nodes[node].get('reg_2')}"
            ).add_to(exit_nodes_layer)
        elif graph.nodes[node].get('city_name'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='red',
                fill=True,
                fill_color='red',
                tooltip=str(graph.nodes[node]['city_name'])
            ).add_to(nodes_city_layer)
        elif graph.nodes[node].get('rr_name'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='yellow',
                fill=True,
                fill_color='yellow',
                tooltip=str(graph.nodes[node]['rr_name'])
            ).add_to(nodes_rr_layer)
        elif graph.nodes[node].get('reg_1') == True or graph.nodes[node].get('reg_2'):
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color='gray',
                fill=True,
                fill_color='gray'
            ).add_to(nodes_default_layer)

# Добавление слоев на карту
nodes_rr_layer.add_to(m)
lines_layer_reg1.add_to(m)
lines_layer_reg2.add_to(m)
exit_nodes_layer.add_to(m)
nodes_city_layer.add_to(m)
nodes_default_layer.add_to(m)

# Добавление контроля слоев
folium.LayerControl().add_to(m)

# Сохранение карты
m.save('../data/html/LO_all_data.html')

## Выделение подграфа (каркаса)

In [ ]:
e = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') in([1, 2])]
subgraph = graph.edge_subgraph(e).copy()

## Assessment of the state of the territory

In [ ]:
# Загрузка локального графа
graph = nx.read_graphml('data/graphml/graph.graphml')
# Перевод nodes:str -> int, geometry:str -> geometry
graph = nx.convert_node_labels_to_integers(graph)
graph = graphbuilder_v2.convert_geometry_from_wkt(graph)
# Перевод edges:str -> list
graph = graphbuilder_v2.convert_list_attr_from_str(graph)

inter = nx.read_graphml('data/graphml/inter.graphml')
inter = prepare_graph(inter)

In [ ]:
# city_points = gpd.read_file("data/geojson/admin_centers_LO_188_points.geojson")
polygons188 = gpd.read_file("data/geojson/polygons188.geojson")
points = gpd.read_file("data/geojson/points.geojson")

p1 = gpd.read_file('data/geojson/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('data/geojson/Аэродром Сиверск .geojson')
p3 = gpd.read_file('data/geojson/project Светогорского поселения.geojson')
p4 = gpd.read_file('data/geojson/project Шлиссельбург.geojson')

In [ ]:
ferry = gpd.read_file('data/geojson/water_transport_LO.geojson')
aero = gpd.read_file('data/geojson/airports_local_LO.geojson')
r_stops = gpd.read_file('data/geojson/ЖД остановки.geojson')
fuel = gpd.read_file('data/geojson/fuel.geojson')
b_stops = gpd.read_file('data/geojson/Остановки ЛО.geojson')

In [ ]:
adj_mx = availability_matrix(graph,points)
p = find_median(points,adj_mx)

In [ ]:
import pandas as pd

p_agg = p[p['to_service'] < np.finfo(np.float64).max].copy()
res = gpd.sjoin(p_agg, polygons188, how='left', op='within').groupby('index_right').median(['to_service']).reset_index()
result_df = pd.merge(polygons188.reset_index(), res, left_on='index', right_on='index_right', how='left').drop(columns=['fid_right']).rename(columns={'to_service': 'in_car'})
result_df = result_df.drop(columns=['index_right'])

In [ ]:
inter = nx.convert_node_labels_to_integers(inter)
adj_mx_inter = availability_matrix(inter,points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])
p_inter = find_median(points,adj_mx_inter)
points_inter = p_inter[p_inter['to_service'] < np.finfo(np.float64).max].copy()


In [ ]:
res_inter = gpd.sjoin(points_inter, polygons188, how="left", predicate="within").groupby('index_right').median(['to_service']).reset_index()
result_df_inter = pd.merge(result_df, res_inter, left_on='index', right_on='index_right', how='left').drop(columns=['index_right', 'fid_right']).rename(columns={'to_service': 'in_inter'})

In [ ]:
neudobiya = pd.concat([p1,p2,p3,p4])
n_grade = grade_territory(neudobiya, graph, r_stops)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import networkx as nx

def weight_territory(gdf, r_stops, b_stops, ferry, aero):
    """
    Grades territories based on their distances to stops, fuel, ferry, and aero points.

    Parameters:
        gdf_poly (GeoDataFrame): A GeoDataFrame containing the polygons of the territories to be graded.
        stops (GeoDataFrame): A GeoDataFrame containing the locations of railway stops.
        fuel (GeoDataFrame): A GeoDataFrame containing the locations of fuel stations.
        ferry (GeoDataFrame): A GeoDataFrame containing the locations of ferry stops.
        aero (GeoDataFrame): A GeoDataFrame containing the locations of airports.

    Returns:
        GeoDataFrame: A GeoDataFrame containing the graded territories with added 'weight' column.
    """
    # Ensure all GeoDataFrames have the same CRS
    common_crs = 32636
    gdf_poly = gdf.copy()
    gdf_poly = gdf_poly.to_crs(epsg=common_crs)
    r_stops = r_stops.to_crs(epsg=common_crs)
    b_stops = b_stops.to_crs(epsg=common_crs)
    ferry = ferry.to_crs(epsg=common_crs)
    aero = aero.to_crs(epsg=common_crs)

    # Define maximum distance
    max_distance = 15000  # 15 km

    # Calculate nearest distances and weights for each type of point
    nearest_r_stops = gdf_poly.sjoin_nearest(r_stops, distance_col='distance_rstops')
    nearest_b_stops = gdf_poly.sjoin_nearest(b_stops, distance_col='distance_bstops')
    nearest_ferry = gdf_poly.sjoin_nearest(ferry, distance_col='distance_ferry')
    nearest_aero = gdf_poly.sjoin_nearest(aero, distance_col='distance_aero')

    # Initialize weights
    gdf_poly['weight'] = 0.0

    # Calculate weights based on distances
    gdf_poly['weight'] += nearest_r_stops['distance_rstops'].apply(lambda x: 0.35 if x <= max_distance else 0.0)
    gdf_poly['weight'] += nearest_b_stops['distance_bstops'].apply(lambda x: 0.35 if x <= max_distance else 0.0)
    gdf_poly['weight'] += nearest_ferry['distance_ferry'].apply(lambda x: 0.20 if x <= max_distance else 0.0)
    gdf_poly['weight'] += nearest_aero['distance_aero'].apply(lambda x: 0.10 if x <= max_distance else 0.0)

    # Ensure weight does not exceed 1.0
    gdf_poly['weight'] = gdf_poly['weight'].apply(lambda x: min(x, 1.0))

    return gdf_poly

# Grade the territories
graded_gdf = weight_territory(n_grade, r_stops, b_stops, ferry, aero)
graded_gdf


In [ ]:
def calculate_quartiles(df, column):
    """Calculate quartile ranks (1 to 4) for a given column in a DataFrame."""
    return pd.qcut(df[column], q=4, labels=False) + 1

def assign_grades(graded_gdf, result_df_inter):
    # Ensure both GeoDataFrames have the same CRS
    result_df_inter = result_df_inter.to_crs(epsg=32636)
    result_df_inter['geom'] = result_df_inter['geometry']
    graded_gdf = graded_gdf.to_crs(epsg=32636)

    # Calculate quartile ranks for 'in_car' and 'in_inter'
    result_df_inter['q_car'] = calculate_quartiles(result_df_inter, 'in_car')
    result_df_inter['q_inter'] = calculate_quartiles(result_df_inter, 'in_inter')
    # Spatial join to find the matching polygons
    joined = gpd.sjoin(graded_gdf, result_df_inter, how="left", predicate='intersects')

    # Calculate the area of intersection
    joined['intersection_area'] = joined.apply(
        lambda row: row['geometry'].intersection(result_df_inter.loc[row['index_right'], 'geometry']).area, axis=1
    )

    # Sort by intersection area and drop duplicates to keep only the max area
    joined = joined.sort_values('intersection_area', ascending=False).drop_duplicates(subset='geometry')
    joined.reset_index(drop=True, inplace=True)

    # Initialize columns for car and public transport grades
    joined['car_grade'] = 0
    joined['public_grade'] = 0
    # Define the grade table
    grade_car = {
        5:   { 'Q4': 2, 'Q3': 3, 'Q2': 4, 'Q1': 5 },
        4.5: { 'Q4': 2, 'Q3': 3, 'Q2': 4, 'Q1': 5 },
        4:   { 'Q4': 1, 'Q3': 2, 'Q2': 3, 'Q1': 4 },
        3.5: { 'Q4': 1, 'Q3': 2, 'Q2': 3, 'Q1': 4 },
        3:   { 'Q4': 0, 'Q3': 1, 'Q2': 3, 'Q1': 3 },
        2.5: { 'Q4': 0, 'Q3': 1, 'Q2': 2, 'Q1': 3 },
        2:   { 'Q4': 0, 'Q3': 1, 'Q2': 2, 'Q1': 2 },
        1.5: { 'Q4': 0, 'Q3': 0, 'Q2': 1, 'Q1': 2 },
        1:   { 'Q4': 0, 'Q3': 0, 'Q2': 1, 'Q1': 1 },
        0:   { 'Q4': 0, 'Q3': 0, 'Q2': 0, 'Q1': 1 },
    }

    grade_public = {
        5:   { 'Q4': 2, 'Q3': 3, 'Q2': 4, 'Q1': 5 },
        4.5: { 'Q4': 2, 'Q3': 3, 'Q2': 4, 'Q1': 5 },
        4:   { 'Q4': 2, 'Q3': 3, 'Q2': 4, 'Q1': 5 },
        3.5: { 'Q4': 1, 'Q3': 2, 'Q2': 3, 'Q1': 5 },
        3:   { 'Q4': 1, 'Q3': 2, 'Q2': 3, 'Q1': 4 },
        2.5: { 'Q4': 1, 'Q3': 2, 'Q2': 3, 'Q1': 4 },
        2:   { 'Q4': 0, 'Q3': 1, 'Q2': 2, 'Q1': 4 },
        1.5: { 'Q4': 0, 'Q3': 1, 'Q2': 2, 'Q1': 3 },
        1:   { 'Q4': 0, 'Q3': 0, 'Q2': 1, 'Q1': 3 },
        0:   { 'Q4': 0, 'Q3': 0, 'Q2': 1, 'Q1': 2 },
    }

    # Apply grades based on the quartiles and grade
    for idx, row in joined.iterrows():
        grade = row['grade']
        car_quartile = row['q_car']
        public_quartile = row['q_inter']
        car_grade = grade_car.get(grade, {}).get(f'Q{car_quartile}', 0)
        public_grade = grade_public.get(grade, {}).get(f'Q{public_quartile}', 0) * row['weight']

        joined.at[idx, 'car_grade'] = car_grade
        joined.at[idx, 'public_grade'] = public_grade
    joined['assessment'] = (joined['car_grade'] + joined['public_grade']) / 2

    return joined[['geometry', 'grade', 'weight', 'q_car', 'q_inter', 'car_grade', 'public_grade', 'assessment']]
result = assign_grades(graded_gdf[['name', 'geometry', 'grade', 'weight']], result_df_inter[['index', 'fid', 'name', 'geometry', 'in_car', 'in_inter']])

In [ ]:
result.to_file('data/geojson/result_assesment.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
import folium
from folium import CircleMarker
from shapely.geometry import Polygon
from pyproj import Transformer
import osmnx as ox

# Инициализация трансформера для преобразования из EPSG:32636 в EPSG:4326
transformer = Transformer.from_crs("epsg:32636", "epsg:4326")

# Преобразование CRS и добавление буферной зоны к полигонам
result = result.to_crs(epsg=32636)
buffered_result = result.copy()
buffered_result['geometry'] = buffered_result['geometry'].buffer(15000)
result = result.to_crs(epsg=4326)  # Возвращаем обратно в WGS 84 для отображения в Folium
buffered_result = buffered_result.to_crs(epsg=4326)

# Функция для добавления CircleMarker в группу слоев
def add_circle_markers(gdf, map_obj, color):
    for idx, row in gdf.iterrows():
        try:
            lat = row.geometry.y
            lon = row.geometry.x
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color=color,
                fill=True,
                fill_color=color,
            ).add_to(map_obj)
        except AttributeError as e:
            print(f"Ошибка на индексе {idx}: {e}")
        except Exception as e:
            print(f"Неизвестная ошибка на индексе {idx}: {e}")

# Создание карты Folium
m = folium.Map(location=[60, 30], zoom_start=6, tiles=None)

# Преобразование и отображение узлов и линий
# Разделение ребер на два списка по значению reg
edges_reg_1 = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') == 1]
edges_reg_2 = [(u, v, k) for u, v, k, d in graph.edges(data=True, keys=True) if d.get('reg') == 2]

# Создание групп слоев
lines_reg_1_layer = folium.FeatureGroup(name='Lines Reg 1', show=False)
lines_reg_2_layer = folium.FeatureGroup(name='Lines Reg 2', show=False)
nodes_layer = folium.FeatureGroup(name='Nodes', show=False)
exit_nodes_layer = folium.FeatureGroup(name='Exit Nodes', show=False)

# Функция для добавления ребер в слой
def add_edges_to_layer(edges, layer, color):
    for u, v, k in edges:
        start_y, start_x = graph.nodes[u]['y'], graph.nodes[u]['x']
        end_y, end_x = graph.nodes[v]['y'], graph.nodes[v]['x']
        
        # Преобразование координат
        start_lat, start_lon = transformer.transform(start_x, start_y)
        end_lat, end_lon = transformer.transform(end_x, end_y)
        
        # Создание линии
        line_coords = [
            [start_lat, start_lon],
            [end_lat, end_lon]
        ]
        folium.PolyLine(locations=line_coords, color=color).add_to(layer)
        
        # Условие для цвета CircleMarker и создание tooltip
        start_color = '#FE4F19' if graph.nodes[u].get('exit') == 1 else '#13D1FF'
        end_color =   '#FE4F19' if graph.nodes[v].get('exit') == 1 else '#13D1FF'
        
        start_tooltip = f"Exit: {graph.nodes[u].get('exit')}, Reg1: {graph.nodes[u].get('reg_1')}, Reg2: {graph.nodes[u].get('reg_2')}"
        end_tooltip =   f"Exit: {graph.nodes[v].get('exit')}, Reg1: {graph.nodes[v].get('reg_1')}, Reg2: {graph.nodes[v].get('reg_2')}"
        
        # Добавление CircleMarker для начального узла
        folium.CircleMarker(
            location=[start_lat, start_lon],
            radius=2,
            color=start_color,
            fill=True,
            opacity=1,
            fill_color=start_color,
            tooltip=start_tooltip
        ).add_to(exit_nodes_layer if graph.nodes[u].get('exit') == 1 else nodes_layer)
        
        # Добавление CircleMarker для конечного узла
        folium.CircleMarker(
            location=[end_lat, end_lon],
            radius=2,
            color=end_color,
            fill=True,
            opacity=1,
            fill_color=end_color,
            tooltip=end_tooltip
        ).add_to(exit_nodes_layer if graph.nodes[v].get('exit') == 1 else nodes_layer)

# Добавление ребер в соответствующие слои
add_edges_to_layer(edges_reg_1, lines_reg_1_layer, color='#AAFF01')
add_edges_to_layer(edges_reg_2, lines_reg_2_layer, color='#FF01AA')

# Добавление слоев на карту
lines_reg_1_layer.add_to(m)
lines_reg_2_layer.add_to(m)
nodes_layer.add_to(m)
exit_nodes_layer.add_to(m)

# Добавление границы полигона (например, города)
city.boundary.explore(m=m, color='#AA00FF', name='Cities', opacity=0.8, show=False)

# Добавление базовых карт
folium.TileLayer('OpenStreetMap', name='Light Map').add_to(m)
folium.TileLayer('cartodbdark_matter', name='Dark Map').add_to(m)

# Создание групп слоев
ferry_group = folium.FeatureGroup(name='Ferry', show=False).add_to(m)
aero_group = folium.FeatureGroup(name='Airports', show=False).add_to(m)
r_stops_group = folium.FeatureGroup(name='Rail stops', show=False).add_to(m)
b_stops_group = folium.FeatureGroup(name='Bus stops', show=False).add_to(m)
result_group = folium.FeatureGroup(name='Result Polygons', show=False).add_to(m)
buffer_group = folium.FeatureGroup(name='Buffer Polygons', show=False).add_to(m)

# Добавление CircleMarkers в соответствующие группы
add_circle_markers(ferry, ferry_group, 'blue')
add_circle_markers(aero, aero_group, 'red')
add_circle_markers(r_stops, r_stops_group, 'green')
add_circle_markers(b_stops, b_stops_group, '#DEAC80')

# Добавление полигонов и буферной зоны в соответствующие группы с tooltip
folium.GeoJson(result, color='#B5C18E').add_to(result_group)
folium.GeoJson(buffered_result, color='#F7DCB9').add_to(buffer_group)

# Добавление контроля слоев
folium.LayerControl().add_to(m)

legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; <i class="fa fa-circle" style="color:blue"></i> Ferry <br>
     &nbsp; <i class="fa fa-circle" style="color:red"></i> Airports <br>
     &nbsp; <i class="fa fa-circle" style="color:green"></i> Railway stops <br>
     &nbsp; <i class="fa fa-circle" style="color:#DEAC80"></i> Bus stops <br>
     &nbsp; <i class="fa fa-square" style="color:#B5C18E"></i> Result Polygons <br>
     &nbsp; <i class="fa fa-square" style="color:#F7DCB9"></i> Buffer Polygons <br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))
# Сохранение карты в файл
m.save('data/html/map.html')

m
